### Создание меток и фич для обучения модели логистической регрессии с использованием resnet

In [4]:
import os
import numpy as np
from PIL import Image
import torch
import torchvision.models as models
from torchvision.models import resnet18, ResNet18_Weights
import torchvision.transforms as transforms
from tqdm import tqdm

data_root = "C:/Users/User/.cache/kagglehub/datasets/darshanthakare/dog-breeds-image-dataset/versions/1/cropped_dogs"
feature_extractor = models.resnet18(weights=ResNet18_Weights.DEFAULT)
feature_extractor = torch.nn.Sequential(*list(feature_extractor.children())[:-1])  # Удаляем последний слой
feature_extractor.eval()

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

features = []
labels = []
class_names = sorted(os.listdir(data_root))

for class_idx, class_name in enumerate(tqdm(class_names, desc="Processing classes")):
    class_dir = os.path.join(data_root, class_name)
    if not os.path.isdir(class_dir):
        continue
    
    for img_name in os.listdir(class_dir):
        img_path = os.path.join(class_dir, img_name)
        try:
            # Загрузка и преобразование изображения
            img = Image.open(img_path).convert('RGB')
            img_tensor = transform(img).unsqueeze(0)
            
            # Извлечение фич
            with torch.no_grad():
                feature = feature_extractor(img_tensor).squeeze().numpy()
            
            features.append(feature)
            labels.append(class_idx)
        except Exception as e:
            print(f"Error processing {img_path}: {e}")

# Конвертация в numpy массивы
X = np.array(features)
y = np.array(labels)

np.save('dog_features75.npy', X)
np.save('dog_labels75.npy', y)

print(f"\nExtracted features: {X.shape}")
print(f"Extracted labels: {y.shape}")
print(f"Class names: {class_names}")

Processing classes: 100%|██████████| 75/75 [09:51<00:00,  7.89s/it]


Extracted features: (10117, 512)
Extracted labels: (10117,)
Class names: ['airedale', 'american_terrier', 'appenzeller', 'australian_terrier', 'basenji', 'basset_hound', 'beagle', 'bernese_mountain', 'blenheim_spaniel', 'blood_hound', 'bluetick', 'border_collie', 'border_terrier', 'boston_bulldog', 'boxer', 'brittany_spaniel', 'bull_mastiff', 'cairn_terrier', 'cardigan_corgi', 'chesapeake_retriever', 'chihuahua', 'chow', 'cocker_spaniel', 'dandie_terrier', 'english_setter', 'english_sheepdog', 'english_springer', 'entlebucher', 'french_bulldog', 'german_pointer', 'germanshepherd', 'golden_retriever', 'husky', 'irish_setter', 'irish_terrier', 'irish_wolfhound', 'italian_greyhound', 'kelpie', 'labrador', 'lakeland_terrier', 'lhasa', 'malinois', 'maltese', 'miniature_pinscher', 'mix', 'norfolk_terrier', 'norwegian_elkhound', 'norwich_terrier', 'papillon', 'pekinese', 'pembroke', 'pomeranian', 'pug', 'pyrenees', 'redbone', 'rottweiler', 'saluki', 'scottish_terrier', 'shetland_sheepdog', '